Programming Assignment: Метрики качества классификации

Вы научитесь:

вычислять различные меры качества классификации: долю правильных ответов, точность, полноту, AUC-ROC и т.д.
сравнивать алгоритмы классификации при наличии ограничений на точность или полноту

In [30]:
import pandas as pd
import numpy as np
from sklearn import metrics
from __future__ import division

# Загрузите файл classification.csv. 
# В нем записаны истинные классы объектов выборки (колонка true) и ответы некоторого классификатора (колонка pred).

data = pd.read_csv("classification.csv")

# Заполните таблицу ошибок классификации:
# Actual Positive	Actual Negative
# Predicted Positive	TP	FP
# Predicted Negative	FN	TN
# Для этого подсчитайте величины TP, FP, FN и TN согласно их определениям. 
# Например, FP — это количество объектов, имеющих класс 0, но отнесенных алгоритмом к классу 1. 
# Ответ в данном вопросе — четыре числа через пробел.

TPFN = data['true'].value_counts()[1]
FPTN = data['true'].value_counts()[0]

print('TPFN, FPTN : %d %d' % (TPFN, FPTN)) 
TP, FP, FN, TN = 0, 0, 0, 0
for i in xrange(data.shape[0]):
    if data['true'][i] == data['pred'][i]:
        if data['true'][i] == 1:
            TP += 1
        if data['true'][i] == 0:
            TN += 1
    else:
        if data['true'][i] == 1:
            FN += 1
        if data['true'][i] == 0:
            FP += 1

print('Answer on question 1: %d %d %d %d' % (TP, FP, FN, TN))

TPFN, FPTN : 102 98
Answer on question 1: 43 34 59 64


In [33]:
# Посчитайте основные метрики качества классификатора:
# Accuracy (доля верно угаданных) — sklearn.metrics.accuracy_score
# Precision (точность) — sklearn.metrics.precision_score
# Recall (полнота) — sklearn.metrics.recall_score
# F-мера — sklearn.metrics.f1_score
# В качестве ответа укажите эти четыре числа через пробел.

accuracy = metrics.accuracy_score(data['true'], data['pred'])
precision = metrics.precision_score(data['true'], data['pred'])
recall = metrics.recall_score(data['true'], data['pred'])
f1 = metrics.f1_score(data['true'], data['pred'])

print('Answer on question 2: %.2f %.2f %.2f %.2f' % (accuracy, precision, recall, f1))

Answer on question 2: 0.54 0.56 0.42 0.48


In [77]:
# Имеется четыре обученных классификатора. В файле scores.csv записаны истинные классы и значения степени принадлежности 
# положительному классу для каждого классификатора на некоторой выборке:
# для логистической регрессии — вероятность положительного класса (колонка score_logreg),
# для SVM — отступ от разделяющей поверхности (колонка score_svm),
# для метрического алгоритма — взвешенная сумма классов соседей (колонка score_knn),
# для решающего дерева — доля положительных объектов в листе (колонка score_tree).
# Загрузите этот файл.

data = pd.read_csv('scores.csv')

# Посчитайте площадь под ROC-кривой для каждого классификатора. 
# Какой классификатор имеет наибольшее значение метрики AUC-ROC (укажите название столбца)? 
# Воспользуйтесь функцией sklearn.metrics.roc_auc_score.

roc_auc_score_logreg = metrics.roc_auc_score(data['true'], data['score_logreg'])
roc_auc_score_svm = metrics.roc_auc_score(data['true'], data['score_svm'])
roc_auc_score_knn = metrics.roc_auc_score(data['true'], data['score_knn'])
roc_auc_score_tree = metrics.roc_auc_score(data['true'], data['score_tree'])
print('roc_auc_score_logreg = %.2f' % roc_auc_score_logreg)
print('roc_auc_score_svm = %.2f' % roc_auc_score_svm)
print('roc_auc_score_knn = %.2f' % roc_auc_score_knn)
print('roc_auc_score_tree = %.2f' % roc_auc_score_tree)

maximum = np.max([roc_auc_score_logreg, roc_auc_score_svm, roc_auc_score_knn, roc_auc_score_tree])
print('max score value = %.2f' % maximum)

# Общее решение
roc_auc_scores = {}
for clf in data.columns[1:]:
    roc_auc_scores[clf] = metrics.roc_auc_score(data['true'], data[clf])
        
print('Answer on question 3: %s' % pd.Series(roc_auc_scores).sort_values(ascending=False).head(1).index[0])


roc_auc_score_logreg = 0.72
roc_auc_score_svm = 0.71
roc_auc_score_knn = 0.64
roc_auc_score_tree = 0.69
max score value = 0.72
Answer on question 3: score_logreg


In [79]:
# Какой классификатор достигает наибольшей точности (Precision) при полноте (Recall) не менее 70% ?

# Найдите максимальной значение точности среди тех записей, для которых полнота не меньше, чем 0.7.
pr_scores = {}
for clf in data.columns[1:]:
    precision, recall, thresholds = metrics.precision_recall_curve(data['true'], data[clf])
    pr_scores[clf] = precision[recall >= 0.7].max()
    
print('Answer on question 4: %s' % pd.Series(pr_scores).sort_values(ascending=False).head(1).index[0])

Answer on question 4: score_tree
